In [7]:
import os
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "Users/ashu/.credentials/service_credentials.json"
os.environ['OPENAI_API_KEY']=''

In [1]:
from langchain.document_loaders import GoogleDriveLoader
loader = GoogleDriveLoader(
        folder_id='15qPzQITIEk6LSDjaf93VHQtAcqS7GTVu',
        recursive=False,
    )
data = loader.load()

In [16]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 0)
all_splits = text_splitter.split_documents(data)

In [17]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma

vectorstore = Chroma.from_documents(documents=all_splits, embedding=OpenAIEmbeddings())

In [20]:
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

In [23]:
from langchain.chains import ConversationalRetrievalChain
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate

template = """Use the following pieces of context to answer the question at the end. 
If you don't know the answer, just say that you don't know, don't try to make up an answer. 
Use three sentences maximum and keep the answer as concise as possible. 
Always say "thanks for asking!" at the end of the answer. 
{context}
Question: {question}
Helpful Answer:"""
question = "where did ashutosh study"

QA_CHAIN_PROMPT = PromptTemplate.from_template(template)

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)
retriever = vectorstore.as_retriever()
chat = ConversationalRetrievalChain.from_llm(llm, retriever=retriever, memory=memory)
result = chat({"question": question})
result["answer"]

'Ashutosh studied at the Indian Institute of Information Technology, Allahabad.'